# Atividade MapReduce

Considerando o dataset detalhado a seguir, extraia o conjunto de informações solicitadas.

### Dataset dados de operacoes comerciais entre paises
- Dados relativos a transacoes comerciais entre paises ao longo do tempo
- 2 bases:
  - base.csv ~433MB (3M de eventos)
  - base_sample.csv ~4MB (35k de eventos)
- 4.4M de instâncias

### Formato da base

|  # | Nome do campo        	| Descrição                                     	|
|----|----------------------	|-----------------------------------------------	|
|  0 | 	País|	País envolvido na transação comercial|
|  1 | 	Ano|	Ano em que a transação foi efetuada|
|  2 | 	Código|	Código da mercadoria|
|  3 | 	Mercadoria| 	Descrição da mercadoria|
|  4 | 	Fluxo|	Fluxo, e.g. Exportação ou Importação|
|  5 | 	Valor|	Valor em dólares|
|  6 | 	Peso|	Peso da mercadoria|
|  7 | 	Unidade| 	Unidade de medida da mercadoria, e.g. Quantidade de itens|
|  8 | 	Quantidade|	Quantidade conforme a unidade especificada da mercadoria|
|  9 | 	Categoria|	Categoria da mercadoria, e.g. Produto Animal|


### Informações a serem extraídas:

1.	País com a maior quantidade de transações comerciais efetuadas;
2.	Mercadoria com a maior quantidade de transações comerciais no Brasil (como a base de dados está em inglês utilize Brazil, com Z, utilize a função “contains” do Java);
3.	Quantidade de transações financeiras realizadas por ano;
4.	Mercadoria com maior quantidade de transações financeiras;
5.	Mercadoria com maior quantidade de transações financeiras em 2016;
6.	Mercadoria com maior quantidade de transações financeiras em 2016, no Brasil (como a base de dados está em inglês utilize Brazil, com Z);
7.	Mercadoria com maior total de peso, de acordo com todas transações comerciais;
8.	Mercadoria com maior total de peso, de acordo com todas transações comerciais, separadas de acordo com o ano;
9.	Média de peso por mercadoria, separadas de acordo com o ano;
10.	Média de peso por mercadoria comercializadas no Brasil (como a base de dados está em inglês utilize Brazil, com Z), separadas de acordo com o ano;
11.	Média de peso por mercadoria comercializadas no Brasil (como a base de dados está em inglês utilize Brazil, com Z), em relação ao fluxo, separadas de acordo com o ano;
12.	Preco medio das commodities de acordo com o ano
13.	Valores maximos, minimos e médios de cada tipo de mercadoria por ano
14.	Pais com o maior preco da commodity em fluxos do tipo Export
15.	Quantidade de transações comerciais de acordo com o fluxo, de acordo com o ano;

In [ ]:
#Instala ambiente local do MapReduce
!pip --quiet install mrjob
!pip --quiet install --upgrade --no-cache-dir gdown
#Efetua download da base completa (400MB)  https://drive.google.com/file/d/1LdeDR5wKP15kUywI7491OmKzdMzVgO-c/view?usp=sharing
!gdown 1LdeDR5wKP15kUywI7491OmKzdMzVgO-c
#Efetua download da base parcial (4MB)   https://drive.google.com/file/d/1CBi-jpBlJrKX4BmWMO0bM-DjLeLXrnDm/view?usp=sharing
!gdown 1CBi-jpBlJrKX4BmWMO0bM-DjLeLXrnDm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.6/439.6 kB 5.7 MB/s eta 0:00:00
Downloading...
From (original): https://drive.google.com/uc?id=1LdeDR5wKP15kUywI7491OmKzdMzVgO-c
From (redirected): https://drive.google.com/uc?id=1LdeDR5wKP15kUywI7491OmKzdMzVgO-c&confirm=t&uuid=90756c4b-22a8-44ee-afad-9b93f82d9f78
To: /content/base.csv
100% 444M/444M [00:02<00:00, 149MB/s]
Downloading...
From: https://drive.google.com/uc?id=1CBi-jpBlJrKX4BmWMO0bM-DjLeLXrnDm
To: /content/base_sample.csv
100% 4.06M/4.06M [00:00<00:00, 131MB/s]


### Informação 1
 - País com a maior quantidade de transações comerciais efetuadas;

In [ ]:
## SELECT PAIS, COUNT(*) FROM tabela GROUP BY PAIS SORT BY COUNT(*) DESC LIMIT 1

In [ ]:
%%file pratica.py
from mrjob.job import MRJob
import sys

class WordCount(MRJob):
    def mapper(self, _, linha):
      try:
        campos = linha.split(';')
        pais = campos[0]
        yield(pais, 1)
      except:
        pass

    def reducer(self, chave, valores):
      valor = 0
      for v in valores:
        valor = valor + 1
      yield(chave, valor)

if __name__ == '__main__':
     WordCount.run()

Writing pratica.py


Teste prévio com base parcial (Top 1 = "Canada"	936)

In [ ]:
!rm -rf /content/atividade
!python pratica.py /content/base_sample.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/pratica.root.20240808.002626.633973
job output is in /content/atividade
Removing temp directory /tmp/pratica.root.20240808.002626.633973...

Top 10 Entradas da atividade
"Canada"	936
"France"	762
"Germany"	740
"Netherlands"	717
"Greece"	687
"Italy"	655
"Denmark"	646
"Czech Rep."	643
"Malaysia"	640
"Austria"	614


Teste com base completa (Top 1 = "Australia"	89487)

In [ ]:
!rm -rf /content/atividade
!python pratica.py /content/base.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/pratica.root.20240808.002634.698382
job output is in /content/atividade
Removing temp directory /tmp/pratica.root.20240808.002634.698382...

Top 10 Entradas da atividade
"Australia"	89487
"Canada"	69468
"Austria"	58683
"Argentina"	57729
"China, Hong Kong SAR"	55535
"Brazil"	54356
"Denmark"	49905
"Belgium"	49525
"China"	49275
"France"	46052


### Informação 2
 - Mercadoria com a maior quantidade de transações comerciais no Brasil (como a base de dados está em inglês utilize Brazil, com Z)

In [ ]:
%%file pratica.py
from mrjob.job import MRJob
import sys

class WordCount(MRJob):
    def mapper(self, _, linha):
      try:
        campos = linha.split(';')
        pais = campos[0]
        mercadoria = campos[3]
        if pais == 'Brazil':
          yield(mercadoria,1)
      except:
        pass

    def reducer(self, chave, valores):
      valor = 0
      for i in valores:
        valor = valor + i
      yield(chave, valor)

if __name__ == '__main__':
     WordCount.run()

Overwriting pratica.py


Teste prévio com base parcial

In [ ]:
!rm -rf /content/atividade
!python pratica.py /content/base_sample.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/pratica.root.20240808.002717.551859
job output is in /content/atividade
Removing temp directory /tmp/pratica.root.20240808.002717.551859...

Top 10 Entradas da atividade
"Horses, live except pure-bred breeding"	64
"Animals, live, except farm animals"	56
"Bovine animals, live pure-bred breeding"	56
"Fowls, live domestic < 185 grams"	56
"Bovine animals, live, except pure-bred breeding"	54
"Horses, live pure-bred breeding"	54
"Swine, live pure-bred breeding"	52
"Sheep, live"	47
"Poultry, live except domestic fowls, < 185 grams"	32
"Goats, live"	30


Teste com base completa

In [ ]:
!rm -rf /content/atividade
!python pratica.py /content/base.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

In [ ]:
## SELECT Mercadoria, COUNT(*) FROM Tabela WHERE Pais = BR GROUP BY Mercadoria

## entradas de filtro:
  # Pais = BR
## campos de filtro:
  # Mercadoria
## Chave: Mercadoria
## valor: ocorrencia

### Informação 3
 - Quantidade de transações financeiras realizadas por ano;

In [ ]:
## select ano, count(*) from tabela group by ano

%%file pratica.py
from mrjob.job import MRJob
import sys

class WordCount(MRJob):
  def mapper(self,_,linha):
    try:
      campos = linha.split(';')
      ano = campos[1]
      yield(ano,1)
    except:
        pass

  def reducer(self, chave, valores):
    yield(chave, sum(list(valores)))

if __name__ == '__main__':
     WordCount.run()

Overwriting pratica.py


Teste prévio com base parcial

In [ ]:
!rm -rf /content/atividade
!python pratica.py /content/base_sample.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/pratica.root.20240808.003633.440429
job output is in /content/atividade
Removing temp directory /tmp/pratica.root.20240808.003633.440429...

Top 10 Entradas da atividade
"2012"	1628
"2013"	1621
"2003"	1580
"2010"	1571
"2014"	1568
"2011"	1566
"2001"	1557
"2015"	1551
"2000"	1549
"2007"	1538


Teste com base completa

In [ ]:
!rm -rf /content/atividade
!python pratica.py /content/base.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/pratica.root.20240808.003643.471882
job output is in /content/atividade
Removing temp directory /tmp/pratica.root.20240808.003643.471882...

Top 10 Entradas da atividade
"2012"	137457
"2011"	137418
"2010"	137198
"2009"	136645
"2006"	135996
"2007"	135855
"2013"	135501
"2005"	133867
"2014"	133270
"2008"	133210


### Informação 4
 - Mercadoria com maior quantidade de transações financeiras;

In [ ]:
%%file pratica.py
from mrjob.job import MRJob
import sys

class WordCount(MRJob):
    def mapper(self, _, linha):
      campos = linha.split(';')
      mercadoria = campos[3]
      yield(mercadoria, 1)

    def reducer(self, chave, valores):
      yield(chave, sum(list(valores)))

if __name__ == '__main__':
     WordCount.run()

Overwriting pratica.py


Teste prévio com base parcial

In [ ]:
!rm -rf /content/atividade
!python pratica.py /content/base_sample.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/pratica.root.20240808.003904.655985
job output is in /content/atividade
Removing temp directory /tmp/pratica.root.20240808.003904.655985...

Top 10 Entradas da atividade
"Animals, live, except farm animals"	4413
"Fowls, live domestic < 185 grams"	3383
"Horses, live except pure-bred breeding"	3331
"Horses, live pure-bred breeding"	2971
"Bovine animals, live, except pure-bred breeding"	2758
"Poultry, live except domestic fowls, < 185 grams"	2494
"Bovine animals, live pure-bred breeding"	2490
"Sheep, live"	2287
"Goats, live"	2002
"Poultry, live except domestic fowls, > 185 grams"	1991


Teste com base completa

In [ ]:
!rm -rf /content/atividade
!python pratica.py /content/base.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

### Informação 5
 - Mercadoria com maior quantidade de transações financeiras em 2016

In [ ]:
%%file pratica.py
from mrjob.job import MRJob
import sys

class WordCount(MRJob):
    def mapper(self, _, linha):
      try:
        campos = linha.split(';')
        mercadoria = campos[3]
        if campos[1] == '2016':
          yield(mercadoria,1)
      except:
        pass

    def reducer(self, chave, valores):
      yield(chave, sum(list(valores)))

if __name__ == '__main__':
     WordCount.run()

Overwriting pratica.py


Teste prévio com base parcial

In [ ]:
!rm -rf /content/atividade
!python pratica.py /content/base_sample.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/pratica.root.20240808.004130.469854
job output is in /content/atividade
Removing temp directory /tmp/pratica.root.20240808.004130.469854...

Top 10 Entradas da atividade
"Animals, live, except farm animals"	161
"Horses, live except pure-bred breeding"	121
"Fowls, live domestic < 185 grams"	119
"Horses, live pure-bred breeding"	107
"Bovine animals, live, except pure-bred breeding"	98
"Bovine animals, live pure-bred breeding"	89
"Sheep, live"	86
"Swine, live pure-bred breeding"	71
"Goats, live"	70
"Poultry, live except domestic fowls, > 185 grams"	69


Teste com base completa

In [ ]:
!rm -rf /content/atividade
!python pratica.py /content/base.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

### Informação 6
 - Mercadoria com maior quantidade de transações financeiras em 2016, no Brasil (como a base de dados está em inglês utilize Brazil, com Z);

In [ ]:
%%file pratica.py
from mrjob.job import MRJob
import sys

class WordCount(MRJob):
    def mapper(self, _, linha):
      try:
        campos = linha.split(';')
        mercadoria = campos[3]
        if campos[1] == '2016' and campos[0] == 'Brazil':
          yield(mercadoria,1)
      except:
        pass

    def reducer(self, chave, valores):
      yield(chave, sum(list(valores)))

if __name__ == '__main__':
     WordCount.run()

Overwriting pratica.py


Teste prévio com base parcial

In [ ]:
!rm -rf /content/atividade
!python pratica.py /content/base_sample.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/pratica.root.20240808.004339.743624
job output is in /content/atividade
Removing temp directory /tmp/pratica.root.20240808.004339.743624...

Top 10 Entradas da atividade
"Animals, live, except farm animals"	2
"Bovine animals, live, except pure-bred breeding"	2
"Bovine animals, live pure-bred breeding"	2
"Fowls, live domestic < 185 grams"	2
"Horses, live except pure-bred breeding"	2
"Horses, live pure-bred breeding"	2
"Swine, live pure-bred breeding"	2
"Fowls, live domestic > 185 grams"	1
"Sheep, live"	1
"Swine, live except pure-bred breeding < 50 kg"	1


Teste com base completa

In [ ]:
!rm -rf /content/atividade
!python pratica.py /content/base.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

### Informação 7
 - Mercadoria com maior total de peso, de acordo com todas transações comerciais;

In [ ]:
%%file pratica.py
from mrjob.job import MRJob
import sys

class WordCount(MRJob):
    def mapper(self, _, linha):
      try:
        campos = linha.split(';')
        mercadoria = campos[3]
        peso = int(campos[6])
        yield(mercadoria,peso)
      except:
          pass

    def reducer(self, chave, valores):
      yield(chave, sum(list(valores)))

if __name__ == '__main__':
     WordCount.run()

Overwriting pratica.py


Teste prévio com base parcial

In [ ]:
!rm -rf /content/atividade
!python pratica.py /content/base_sample.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/pratica.root.20240808.004612.828791
job output is in /content/atividade
Removing temp directory /tmp/pratica.root.20240808.004612.828791...

Top 10 Entradas da atividade
"Bovine animals, live, except pure-bred breeding"	72350442141
"Swine, live except pure-bred breeding > 50 kg"	38798669146
"Fowls, live domestic > 185 grams"	18665789461
"Sheep, live"	13771569405
"Swine, live except pure-bred breeding < 50 kg"	12228873816
"Bovine animals, live pure-bred breeding"	7733896254
"Poultry, live except domestic fowls, > 185 grams"	4546972430
"Horses, live except pure-bred breeding"	2396204313
"Animals, live, except farm animals"	1469485365
"Fowls, live domestic < 185 grams"	1404224958


Teste com base completa

In [ ]:
!rm -rf /content/atividade
!python pratica.py /content/base.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

### Informação 8
 - Mercadoria com maior total de peso, de acordo com todas transações comerciais, separadas de acordo com o ano;

In [ ]:
%%file pratica.py
from mrjob.job import MRJob
import sys

class WordCount(MRJob):
    def mapper(self, _, linha):
      try:
        campos = linha.split(';')
        ano = campos[1]
        mercadoria = campos[3]
        chave = ",".join([mercadoria, ano])
        peso = int(campos[6])
        yield(chave,peso)
      except:
        pass

    def reducer(self, chave, valores):
      yield(chave, sum(list(valores)))

if __name__ == '__main__':
     WordCount.run()

Overwriting pratica.py


Teste prévio com base parcial

In [ ]:
!rm -rf /content/atividade
!python pratica.py /content/base_sample.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/pratica.root.20240808.005216.801181
job output is in /content/atividade
Removing temp directory /tmp/pratica.root.20240808.005216.801181...

Top 10 Entradas da atividade
"Bovine animals, live, except pure-bred breeding,2011"	3790561468
"Bovine animals, live, except pure-bred breeding,2010"	3725385865
"Bovine animals, live, except pure-bred breeding,2009"	3667337654
"Bovine animals, live, except pure-bred breeding,2007"	3582928269
"Bovine animals, live, except pure-bred breeding,2012"	3532005526
"Bovine animals, live, except pure-bred breeding,2006"	3474206442
"Bovine animals, live, except pure-bred breeding,2008"	3438256284
"Bovine animals, live, except pure-bred breeding,2013"	3438146811
"Bovine animals, live, except pure-bred breeding,2015"	3430878298
"Bovine animals, live, except pure-bred breeding,2005"	3358493111


Teste com base completa

In [ ]:
!rm -rf /content/atividade
!python pratica.py /content/base.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

### Informação 9
 - Média de peso por mercadoria, separadas de acordo com o ano;

In [ ]:
%%file pratica.py
from mrjob.job import MRJob
import sys

class WordCount(MRJob):
    def mapper(self, _, linha):
      try:
        campos = linha.split(';')
        ano = campos[1]
        mercadoria = campos[3]
        chave = mercadoria+ ano
        peso = int(campos[6])
        yield(chave,peso)
      except:
        pass

    def reducer(self, chave, valores):
      lista = list(valores)
      yield(chave, (sum(lista)/len(lista)))

if __name__ == '__main__':
     WordCount.run()

Overwriting pratica.py


Teste prévio com base parcial

In [ ]:
!rm -rf /content/atividade
!python pratica.py /content/base_sample.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/pratica.root.20240808.010216.813549
job output is in /content/atividade
Removing temp directory /tmp/pratica.root.20240808.010216.813549...

Top 10 Entradas da atividade
"Sheep, live1988"	54003615.0
"Swine, live except pure-bred breeding > 50 kg2008"	43181538.152542375
"Swine, live except pure-bred breeding > 50 kg2011"	36794547.88888889
"Swine, live except pure-bred breeding > 50 kg2013"	35665136.307692304
"Swine, live except pure-bred breeding > 50 kg1988"	35478237.5
"Swine, live except pure-bred breeding > 50 kg2014"	34317240.484375
"Fowls, live domestic > 185 grams2016"	33725574.385964915
"Swine, live except pure-bred breeding > 50 kg2007"	32519872.38596491
"Swine, live except pure-bred breeding > 50 kg2016"	32402963.825396825
"Swine, live except pure-bred breeding > 50 kg2009"	32227585.830985915


Teste com base completa

In [ ]:
!rm -rf /content/atividade
!python pratica.py /content/base.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

### Informação 10
 - Média de peso por mercadoria comercializadas no Brasil (como a base de dados está em inglês utilize Brazil, com Z), separadas de acordo com o ano;

In [ ]:
%%file pratica.py
from mrjob.job import MRJob
import sys

class WordCount(MRJob):
    def mapper(self, _, linha):
      try:
        campos = linha.split(';')
        ano = campos[1]
        mercadoria = campos[3]
        chave = mercadoria+ ano
        peso = int(campos[6])
        if campos[0] == 'Brazil':
          yield(chave,peso)
      except:
        pass

    def reducer(self, chave, valores):
      lista = list(valores)
      yield(chave, (sum(lista)/len(lista)))

if __name__ == '__main__':
     WordCount.run()

Overwriting pratica.py


Teste prévio com base parcial

In [ ]:
!rm -rf /content/atividade
!python pratica.py /content/base_sample.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/pratica.root.20240808.011201.794632
job output is in /content/atividade
Removing temp directory /tmp/pratica.root.20240808.011201.794632...

Top 10 Entradas da atividade
"Bovine animals, live, except pure-bred breeding2010"	163259108.0
"Bovine animals, live, except pure-bred breeding2013"	161980315.0
"Bovine animals, live, except pure-bred breeding2014"	151681448.5
"Bovine animals, live, except pure-bred breeding2009"	134964128.0
"Bovine animals, live, except pure-bred breeding2012"	121096675.0
"Bovine animals, live, except pure-bred breeding2008"	105512865.5
"Bovine animals, live, except pure-bred breeding2007"	102115793.5
"Bovine animals, live, except pure-bred breeding2011"	98217470.0
"Bovine animals, live, except pure-bred breeding1995"	63394422.0
"Bovine animals, live, except pure-bred breeding1994"	52523980.0


Teste com base completa

In [ ]:
!rm -rf /content/atividade
!python pratica.py /content/base.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

### Informação 11
 - Média de peso por mercadoria comercializadas no Brasil (como a base de dados está em inglês utilize Brazil, com Z), em relação ao fluxo, separadas de acordo com o ano;

In [ ]:
%%file pratica.py
from mrjob.job import MRJob
import sys

class WordCount(MRJob):
    def mapper(self, _, linha):
      try:
        campos = linha.split(';')
        ano = campos[1]
        mercadoria = campos[3]
        fluxo = campos[4]
        chave = mercadoria+fluxo+ ano
        peso = int(campos[6])
        if campos[0] == 'Brazil':
          yield(chave,peso)
      except:
        pass

    def reducer(self, chave, valores):
      lista = list(valores)
      yield(chave, (sum(lista)/len(lista)))

if __name__ == '__main__':
     WordCount.run()

Overwriting pratica.py


Teste prévio com base parcial

In [ ]:
!rm -rf /content/atividade
!python pratica.py /content/base_sample.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/pratica.root.20240808.011746.630717
job output is in /content/atividade
Removing temp directory /tmp/pratica.root.20240808.011746.630717...

Top 10 Entradas da atividade
"Bovine animals, live, except pure-bred breedingExport2013"	323951180.0
"Bovine animals, live, except pure-bred breedingExport2010"	323732156.0
"Bovine animals, live, except pure-bred breedingExport2014"	298824624.0
"Bovine animals, live, except pure-bred breedingExport2009"	257772216.0
"Bovine animals, live, except pure-bred breedingExport2012"	242159350.0
"Bovine animals, live, except pure-bred breedingExport2007"	199888037.0
"Bovine animals, live, except pure-bred breedingExport2011"	194178940.0
"Bovine animals, live, except pure-bred breedingExport2008"	193144870.0
"Bovine animals, live, except pure-bred breedingImport1995"	126427408.0
"Bovine animals, live, except pure-bre

Teste com base completa

In [ ]:
!rm -rf /content/atividade
!python pratica.py /content/base.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

### Informação 12
 - Preco medio das commodities de acordo com o ano

In [ ]:
%%file pratica.py
from mrjob.job import MRJob
import sys

class WordCount(MRJob):
    def mapper(self, _, linha):
      try:
        campos = linha.split(';')
        ano = campos[1]
        commodities = campos[3]
        fluxo = campos[4]
        chave = commodities+','+ano
        preco = float(campos[5])
        yield(chave,preco)
      except:
        pass

    def reducer(self, chave, valores):
      lista = list(valores)
      yield(chave, (sum(lista)/len(lista)))

if __name__ == '__main__':
     WordCount.run()

Overwriting pratica.py


Teste prévio com base parcial

In [ ]:
!rm -rf /content/atividade
!python pratica.py /content/base_sample.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/pratica.root.20240808.012046.077468
job output is in /content/atividade
Removing temp directory /tmp/pratica.root.20240808.012046.077468...

Top 10 Entradas da atividade
"Bovine animals, live, except pure-bred breeding,2016"	87946336.62244898
"Bovine animals, live, except pure-bred breeding,2014"	87511974.93283582
"Bovine animals, live, except pure-bred breeding,1994"	84802246.29166667
"Bovine animals, live, except pure-bred breeding,2013"	83745374.140625
"Bovine animals, live, except pure-bred breeding,2010"	82026854.73728813
"Bovine animals, live, except pure-bred breeding,2015"	80993941.36
"Bovine animals, live, except pure-bred breeding,2012"	80702915.28148147
"Bovine animals, live, except pure-bred breeding,2011"	79034358.08333333
"Bovine animals, live, except pure-bred breeding,1995"	77522772.4090909
"Swine, live except pure-bred breeding

Teste com base completa

In [ ]:
!rm -rf /content/atividade
!python pratica.py /content/base.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

### Informação 13
 - Valores maximos, minimos e médios de cada tipo de mercadoria por ano

In [ ]:
%%file pratica.py
from mrjob.job import MRJob
import sys

class WordCount(MRJob):
    def mapper(self, _, linha):
      yield('nada', 1)

    def reducer(self, chave, valores):
      yield('nada', 1)

if __name__ == '__main__':
     WordCount.run()

Teste prévio com base parcial

In [ ]:
!rm -rf /content/atividade
!python pratica.py /content/base_sample.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

Teste com base completa

In [ ]:
!rm -rf /content/atividade
!python pratica.py /content/base.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

### Informação 14
 - Pais com o maior preco da commodity em fluxos do tipo Export

In [ ]:
%%file pratica.py
from mrjob.job import MRJob
import sys

class WordCount(MRJob):
    def mapper(self, _, linha):
      try:
        campos = linha.split(';')
        pais = campos[0]
        chave = pais
        preco = float(campos[5])
        if campos[4] =='Export':
          yield(chave,preco)
      except:
        pass

    def reducer(self, chave, valores):
      lista = list(valores)
      yield(chave, (sum(lista)))

if __name__ == '__main__':
     WordCount.run()

Overwriting pratica.py


Teste prévio com base parcial

In [ ]:
!rm -rf /content/atividade
!python pratica.py /content/base_sample.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/pratica.root.20240808.012813.092117
job output is in /content/atividade
Removing temp directory /tmp/pratica.root.20240808.012813.092117...

Top 10 Entradas da atividade
"France"	46285035324.0
"Canada"	38782920986.0
"Netherlands"	35250017181.0
"EU-28"	26309306412.0
"Germany"	25218535717.0
"Australia"	18959375061.0
"Denmark"	12759819798.0
"Mexico"	12599247218.0
"China"	11424557852.0
"Ireland"	8346981506.0


Teste com base completa

In [ ]:
!rm -rf /content/atividade
!python pratica.py /content/base.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

### Informação 15
 - Quantidade de transações comerciais de acordo com o fluxo, de acordo com o ano

In [ ]:
%%file pratica.py
from mrjob.job import MRJob
import sys

class WordCount(MRJob):
    def mapper(self, _, linha):
      yield('nada', 1)

    def reducer(self, chave, valores):
      yield('nada', 1)

if __name__ == '__main__':
     WordCount.run()

Teste prévio com base parcial

In [ ]:
!rm -rf /content/atividade
!python pratica.py /content/base_sample.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

Teste com base completa

In [ ]:
!rm -rf /content/atividade
!python pratica.py /content/base.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10